# Debug Sign Error in Updated Integrator

This notebook isolates the electromagnetic force sign error by:
1. Testing retarded-only integration (no static-to-retarded handoff)
2. Examining nhat vector calculations
3. Checking distance calculations 
4. Investigating chrono_jn behavior
5. Direct force comparison between legacy and updated

In [1]:
# Import Required Libraries
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('/home/benfol/work/LW_windows')

from core.trajectory_integrator import LienardWiechertIntegrator
sys.path.append('legacy')
from covariant_integrator_library import retarded_integrator3
from bunch_inits import init_bunch

print("✅ Debug environment loaded")

✅ Debug environment loaded


In [6]:
# Create Simple Two-Particle Setup for Direct Comparison
def create_simple_two_particle_setup():
    """Create minimal two-particle setup for force analysis."""
    c_mmns = 299.792458
    
    # Particle 1: Light proton (rider)
    mass1 = 1.007319468  # amu
    q1 = 1.0 * 1.178734E-5  # Positive charge
    pz1 = 1.01e6 * mass1  # High momentum
    gamma1 = np.sqrt(1 + (pz1/(mass1*c_mmns))**2)
    
    # Particle 2: Heavy lead (driver) 
    mass2 = 207.2  # amu
    q2 = -54.0 * 1.178734E-5  # Negative charge
    pz2 = -pz1 / 207.2 * 1.007319468  # Momentum conservation
    gamma2 = np.sqrt(1 + (pz2/(mass2*c_mmns))**2)
    
    # FIXED POSITIONS - ensure both integrators use identical geometry
    separation = 0.500000  # mm - fixed separation
    
    # Updated integrator format
    updated_state1 = {
        'x': np.array([0.0]), 'y': np.array([0.0]), 'z': np.array([0.0]), 't': np.array([0.0]),
        'Px': np.array([0.0]), 'Py': np.array([0.0]), 'Pz': np.array([pz1]),
        'Pt': np.array([gamma1 * mass1 * c_mmns]),
        'gamma': np.array([gamma1]),
        'bx': np.array([0.0]), 'by': np.array([0.0]), 'bz': np.array([pz1/(gamma1*mass1*c_mmns)]),
        'bdotx': np.array([0.0]), 'bdoty': np.array([0.0]), 'bdotz': np.array([0.0]),
        'q': np.array([q1]), 'm': np.array([mass1]), 'char_time': np.array([2/3 * q1**2 / (mass1 * c_mmns**3)])
    }
    
    updated_state2 = {
        'x': np.array([separation]), 'y': np.array([0.0]), 'z': np.array([0.0]), 't': np.array([0.0]),
        'Px': np.array([0.0]), 'Py': np.array([0.0]), 'Pz': np.array([pz2]),
        'Pt': np.array([gamma2 * mass2 * c_mmns]),
        'gamma': np.array([gamma2]),
        'bx': np.array([0.0]), 'by': np.array([0.0]), 'bz': np.array([pz2/(gamma2*mass2*c_mmns)]),
        'bdotx': np.array([0.0]), 'bdoty': np.array([0.0]), 'bdotz': np.array([0.0]),
        'q': np.array([q2]), 'm': np.array([mass2]), 'char_time': np.array([2/3 * q2**2 / (mass2 * c_mmns**3)])
    }
    
    # Create legacy states with IDENTICAL positions to updated states
    # Instead of using init_bunch (which randomizes positions), create manually
    legacy_state1 = {
        'x': np.array([0.0]), 'y': np.array([0.0]), 'z': np.array([0.0]), 't': np.array([0.0]),
        'Px': np.array([0.0]), 'Py': np.array([0.0]), 'Pz': np.array([pz1]),
        'Pt': np.array([gamma1 * mass1 * c_mmns]),
        'gamma': np.array([gamma1]),
        'bx': np.array([0.0]), 'by': np.array([0.0]), 'bz': np.array([pz1/(gamma1*mass1*c_mmns)]),
        'bdotx': np.array([0.0]), 'bdoty': np.array([0.0]), 'bdotz': np.array([0.0]),
        'q': q1, 'm': mass1, 'char_time': 2/3 * q1**2 / (mass1 * c_mmns**3)  # Legacy uses scalars
    }
    
    legacy_state2 = {
        'x': np.array([separation]), 'y': np.array([0.0]), 'z': np.array([0.0]), 't': np.array([0.0]),
        'Px': np.array([0.0]), 'Py': np.array([0.0]), 'Pz': np.array([pz2]),
        'Pt': np.array([gamma2 * mass2 * c_mmns]),
        'gamma': np.array([gamma2]),
        'bx': np.array([0.0]), 'by': np.array([0.0]), 'bz': np.array([pz2/(gamma2*mass2*c_mmns)]),
        'bdotx': np.array([0.0]), 'bdoty': np.array([0.0]), 'bdotz': np.array([0.0]),
        'q': q2, 'm': mass2, 'char_time': 2/3 * q2**2 / (mass2 * c_mmns**3)  # Legacy uses scalars
    }
    
    print(f"Setup created with IDENTICAL initial positions:")
    print(f"  Particle 1: q={q1:.8f}, mass={mass1:.3f}, γ={gamma1:.6f}")
    print(f"  Particle 2: q={q2:.8f}, mass={mass2:.3f}, γ={gamma2:.6f}")
    print(f"  Separation: {separation:.6f} mm (FIXED for both integrators)")
    print(f"  Charge product (q1*q2): {q1*q2:.12f} (negative = attractive)")
    
    return (updated_state1, updated_state2), (legacy_state1, legacy_state2)

# Create test setup with identical initial conditions
(updated_p1, updated_p2), (legacy_p1, legacy_p2) = create_simple_two_particle_setup()

Setup created with IDENTICAL initial positions:
  Particle 1: q=0.00001179, mass=1.007, γ=3368.997510
  Particle 2: q=-0.00063652, mass=207.200, γ=1.003165
  Separation: 0.500000 mm (FIXED for both integrators)
  Charge product (q1*q2): -0.000000007503 (negative = attractive)


In [4]:
# Test 1: Pure Retarded Integration (No Static Phase)
print("🔍 TEST 1: Pure Retarded Integration Comparison")
print("="*60)

# Test parameters - use 1 static step to avoid legacy integrator issues
test_params = {
    'static_steps': 1,  # Minimal static phase
    'ret_steps': 3,     # Just 3 retarded steps
    'step_size': 1e-8,  # Very small step
    'sim_type': 2,      # Free particles
    'wall_pos': 1e5, 'aperture': 1e5, 'bunch_dist': 1e5, 'z_cutoff': 0, 'cav_spacing': 1e5
}

print(f"Integration: {test_params['static_steps']} static + {test_params['ret_steps']} retarded steps, h={test_params['step_size']:.0e}")

# Updated integrator test
integrator = LienardWiechertIntegrator()
updated_traj1, updated_traj2 = integrator.integrate_retarded_fields(
    test_params['static_steps'], test_params['ret_steps'], test_params['step_size'],
    test_params['wall_pos'], test_params['aperture'], test_params['sim_type'],
    updated_p1, updated_p2, test_params['bunch_dist'], test_params['z_cutoff'], test_params['cav_spacing']
)

# Legacy integrator test
legacy_traj1, legacy_traj2 = retarded_integrator3(
    test_params['static_steps'], test_params['ret_steps'], test_params['step_size'],
    test_params['wall_pos'], test_params['aperture'], test_params['sim_type'],
    legacy_p1, legacy_p2, test_params['bunch_dist'], test_params['cav_spacing'], test_params['z_cutoff']
)

print(f"\n✅ Integration completed")
print(f"   Updated: {len(updated_traj1)} steps, Legacy: {len(legacy_traj1)} steps")

# Compare gamma evolution - focus on the retarded steps (after static phase)
print(f"\n=== GAMMA EVOLUTION COMPARISON (Retarded Steps Only) ===")
static_steps = test_params['static_steps']

for i in range(static_steps + 1, min(len(updated_traj1), len(legacy_traj1))):  # Start after static phase
    updated_g1 = updated_traj1[i]['gamma'][0]
    updated_g2 = updated_traj2[i]['gamma'][0]
    legacy_g1 = legacy_traj1[i]['gamma'][0]
    legacy_g2 = legacy_traj2[i]['gamma'][0]
    
    # Compare to end of static phase
    updated_dg1 = updated_g1 - updated_traj1[static_steps]['gamma'][0]
    updated_dg2 = updated_g2 - updated_traj2[static_steps]['gamma'][0]
    legacy_dg1 = legacy_g1 - legacy_traj1[static_steps]['gamma'][0]
    legacy_dg2 = legacy_g2 - legacy_traj2[static_steps]['gamma'][0]
    
    step_label = f"Retarded {i-static_steps}"
    print(f"{step_label}: P1 Δγ - Updated: {updated_dg1:+.10f}, Legacy: {legacy_dg1:+.10f}")
    print(f"{'':>12} P2 Δγ - Updated: {updated_dg2:+.10f}, Legacy: {legacy_dg2:+.10f}")
    
    # Check for sign errors
    if abs(updated_dg1) > 1e-12 and abs(legacy_dg1) > 1e-12:
        if np.sign(updated_dg1) != np.sign(legacy_dg1):
            print(f"{'':>12} 🚨 P1 SIGN ERROR: Updated={np.sign(updated_dg1):+.0f}, Legacy={np.sign(legacy_dg1):+.0f}")
    if abs(updated_dg2) > 1e-12 and abs(legacy_dg2) > 1e-12:
        if np.sign(updated_dg2) != np.sign(legacy_dg2):
            print(f"{'':>12} 🚨 P2 SIGN ERROR: Updated={np.sign(updated_dg2):+.0f}, Legacy={np.sign(legacy_dg2):+.0f}")

print("="*60)

🔍 TEST 1: Pure Retarded Integration Comparison
Integration: 1 static + 3 retarded steps, h=1e-08
  Updated integrator: 4 steps (static: 1, retarded: 3)
  Simulation type: 2, wall_Z: 100000.0, apt_R: 100000.0
DEBUG INTEGRATION: Step 0/4, static_steps=1
DEBUG: Initializing step 0
DEBUG INTEGRATION: Step 1/4, static_steps=1
DEBUG: Retarded integration step 1
DEBUG RETARDED METHOD CALLED: particles=1, i_traj=0
DEBUG RETARDED: qi*qj=-0.00000001, k_factor=1.000000, R=0.500000
DEBUG RETARDED: charge_factor=-0.000000000000
DEBUG RETARDED: ΔPx=0.000000000001, ΔPz=-0.000000000000
DEBUG RETARDED METHOD CALLED: particles=1, i_traj=0
DEBUG RETARDED: qi*qj=-0.00000001, k_factor=1.000000, R=0.500000
DEBUG RETARDED: charge_factor=-0.000000000000
DEBUG RETARDED: ΔPx=-0.000000000000, ΔPz=0.000000000000
DEBUG INTEGRATION: Step 2/4, static_steps=1
DEBUG: Retarded integration step 2
DEBUG RETARDED METHOD CALLED: particles=1, i_traj=1
DEBUG RETARDED: qi*qj=-0.00000001, k_factor=1.001603, R=0.500102
DEBUG RE

In [5]:
# Test 2: Detailed Force Vector Analysis
print("🔍 TEST 2: Detailed Distance and Direction Vector Analysis")
print("="*60)

# Manually check the distance calculation and nhat vectors
# This will help identify if the issue is in geometric calculations

def analyze_geometry(state1, state2, label):
    """Analyze geometric relationships between particles."""
    x1, y1, z1 = state1['x'][0], state1['y'][0], state1['z'][0]
    x2, y2, z2 = state2['x'][0], state2['y'][0], state2['z'][0]
    
    # Distance vector (from particle 2 to particle 1)
    dx, dy, dz = x1 - x2, y1 - y2, z1 - z2
    distance = np.sqrt(dx**2 + dy**2 + dz**2)
    
    # Unit vector (direction from 2 to 1)
    if distance > 0:
        nx, ny, nz = dx/distance, dy/distance, dz/distance
    else:
        nx, ny, nz = 0, 0, 0
    
    print(f"{label}:")
    print(f"  P1 position: ({x1:.6f}, {y1:.6f}, {z1:.6f})")
    print(f"  P2 position: ({x2:.6f}, {y2:.6f}, {z2:.6f})")
    print(f"  Distance vector (P2→P1): ({dx:.6f}, {dy:.6f}, {dz:.6f})")
    print(f"  Distance magnitude: {distance:.6f} mm")
    print(f"  Unit vector (nhat): ({nx:.6f}, {ny:.6f}, {nz:.6f})")
    
    # Check charges and expected force direction
    q1, q2 = state1['q'][0], state2['q'][0]
    force_nature = "ATTRACTIVE" if q1 * q2 < 0 else "REPULSIVE"
    print(f"  Charges: q1={q1:.8f}, q2={q2:.8f}, product={q1*q2:.8f}")
    print(f"  Force nature: {force_nature}")
    
    if q1 * q2 < 0:  # Attractive
        print(f"  Expected: P1 accelerates toward P2 (gains energy if approaching)")
        print(f"  Expected: P2 accelerates toward P1 (loses energy if heavier)")
    
    return distance, (nx, ny, nz), q1*q2

# Analyze initial geometry for both systems
print("\n=== INITIAL GEOMETRY ANALYSIS ===")
updated_dist, updated_nhat, updated_qprod = analyze_geometry(updated_p1, updated_p2, "UPDATED")
print()
legacy_dist, legacy_nhat, legacy_qprod = analyze_geometry(legacy_p1, legacy_p2, "LEGACY")

# Check for differences
print(f"\n=== GEOMETRY COMPARISON ===")
print(f"Distance - Updated: {updated_dist:.8f}, Legacy: {legacy_dist:.8f}, Diff: {updated_dist-legacy_dist:.8f}")
print(f"Nhat X - Updated: {updated_nhat[0]:.8f}, Legacy: {legacy_nhat[0]:.8f}, Diff: {updated_nhat[0]-legacy_nhat[0]:.8f}")
print(f"Charge product - Updated: {updated_qprod:.12f}, Legacy: {legacy_qprod:.12f}, Match: {abs(updated_qprod-legacy_qprod)<1e-12}")

print("="*60)

🔍 TEST 2: Detailed Distance and Direction Vector Analysis

=== INITIAL GEOMETRY ANALYSIS ===
UPDATED:
  P1 position: (0.000000, 0.000000, 0.000000)
  P2 position: (0.500000, 0.000000, 0.000000)
  Distance vector (P2→P1): (-0.500000, 0.000000, 0.000000)
  Distance magnitude: 0.500000 mm
  Unit vector (nhat): (-1.000000, 0.000000, 0.000000)
  Charges: q1=0.00001179, q2=-0.00063652, product=-0.00000001
  Force nature: ATTRACTIVE
  Expected: P1 accelerates toward P2 (gains energy if approaching)
  Expected: P2 accelerates toward P1 (loses energy if heavier)

LEGACY:
  P1 position: (0.000000, 0.000000, -0.000000)
  P2 position: (0.208645, 0.219698, 0.000000)
  Distance vector (P2→P1): (-0.208645, -0.219698, -0.000001)
  Distance magnitude: 0.302985 mm
  Unit vector (nhat): (-0.688632, -0.725111, -0.000002)


TypeError: 'float' object is not subscriptable

In [ ]:
# Test 3: Debug chrono_jn and Retardation Time Calculation
print("🔍 TEST 3: Retardation Time (chrono_jn) Analysis")
print("="*60)

# Let's examine what the chrono_jn function is doing
# and how it affects the retarded field calculations

# First, let's look at the integrator's chrono_jn method
integrator = LienardWiechertIntegrator()

# Create a simple trajectory for testing retardation
simple_traj = [updated_p1.copy(), updated_p1.copy()]  # Two identical time points
simple_traj[1]['t'] = np.array([1e-8])  # Second point at t = 1e-8 ns
simple_traj[1]['z'] = np.array([1e-6])  # Moved slightly in z

simple_traj_ext = [updated_p2.copy(), updated_p2.copy()]
simple_traj_ext[1]['t'] = np.array([1e-8])
simple_traj_ext[1]['z'] = np.array([1e-6])

print("Testing chrono_jn with simple trajectory...")

try:
    # Test the chrono_jn function directly
    if hasattr(integrator, 'chrono_jn'):
        i_traj = 1  # Use second time point
        particle_idx = 0
        
        chrono_result = integrator.chrono_jn(
            simple_traj, simple_traj_ext, i_traj, particle_idx
        )
        
        print(f"✅ chrono_jn result: {chrono_result}")
        print(f"   Type: {type(chrono_result)}")
        if hasattr(chrono_result, '__len__'):
            print(f"   Length: {len(chrono_result)}")
            print(f"   Values: {chrono_result}")
    else:
        print("❌ chrono_jn method not found")
        
    # Test dist_euclid_ret function
    if hasattr(integrator, 'dist_euclid_ret'):
        dist_result = integrator.dist_euclid_ret(
            simple_traj, simple_traj_ext, i_traj, particle_idx
        )
        
        print(f"\n✅ dist_euclid_ret result type: {type(dist_result)}")
        if isinstance(dist_result, dict):
            for key, value in dist_result.items():
                print(f"   {key}: {value} (type: {type(value)})")
                if hasattr(value, '__len__') and len(value) > 0:
                    print(f"      First few values: {value[:min(3, len(value))]}")
    else:
        print("❌ dist_euclid_ret method not found")
        
except Exception as e:
    print(f"❌ Error testing retardation functions: {e}")
    import traceback
    traceback.print_exc()

print("="*60)

In [ ]:
# Test 4: Direct Electromagnetic Force Calculation Comparison
print("🔍 TEST 4: Direct Force Calculation Comparison")
print("="*60)

# Let's manually calculate what the electromagnetic force should be
# and compare it to what both integrators are producing

def manual_coulomb_force(q1, q2, r_vec, m1, gamma1):
    """Calculate expected Coulomb force manually."""
    r_mag = np.linalg.norm(r_vec)
    r_hat = r_vec / r_mag
    
    # Basic Coulomb force: F = k * q1 * q2 / r^2 * r_hat
    # In Gaussian units: F = q1 * q2 / r^2 * r_hat
    force_magnitude = q1 * q2 / (r_mag**2)
    force_vector = force_magnitude * r_hat
    
    # Force on particle 1 due to particle 2
    print(f"Manual calculation:")
    print(f"  q1={q1:.8f}, q2={q2:.8f}, r_mag={r_mag:.6f}")
    print(f"  Force magnitude: {force_magnitude:.12f}")
    print(f"  Force vector: ({force_vector[0]:.12f}, {force_vector[1]:.12f}, {force_vector[2]:.12f})")
    print(f"  Direction: {'toward P2' if force_magnitude < 0 else 'away from P2'}")
    
    return force_vector

# Calculate expected force between our test particles
q1, q2 = updated_p1['q'][0], updated_p2['q'][0]
pos1 = np.array([updated_p1['x'][0], updated_p1['y'][0], updated_p1['z'][0]])
pos2 = np.array([updated_p2['x'][0], updated_p2['y'][0], updated_p2['z'][0]])
r_vec = pos1 - pos2  # Vector from P2 to P1
m1, gamma1 = updated_p1['m'][0], updated_p1['gamma'][0]

print("\n=== EXPECTED FORCE CALCULATION ===")
expected_force = manual_coulomb_force(q1, q2, r_vec, m1, gamma1)

# Now let's see what the integrators actually calculated
print(f"\n=== ACTUAL INTEGRATOR RESULTS ===")

# Compare momentum changes (which reflect applied forces)
if len(updated_traj1) > 1 and len(legacy_traj1) > 1:
    # Updated integrator momentum change
    updated_dp1 = np.array([updated_traj1[1]['Px'][0] - updated_traj1[0]['Px'][0],
                           updated_traj1[1]['Py'][0] - updated_traj1[0]['Py'][0],
                           updated_traj1[1]['Pz'][0] - updated_traj1[0]['Pz'][0]])
    
    # Legacy integrator momentum change  
    legacy_dp1 = np.array([legacy_traj1[1]['Px'][0] - legacy_traj1[0]['Px'][0],
                          legacy_traj1[1]['Py'][0] - legacy_traj1[0]['Py'][0],
                          legacy_traj1[1]['Pz'][0] - legacy_traj1[0]['Pz'][0]])
    
    dt = test_params['step_size']
    updated_force = updated_dp1 / dt  # F = dp/dt
    legacy_force = legacy_dp1 / dt
    
    print(f"Updated integrator:")
    print(f"  Δp: ({updated_dp1[0]:.12f}, {updated_dp1[1]:.12f}, {updated_dp1[2]:.12f})")
    print(f"  Force: ({updated_force[0]:.12f}, {updated_force[1]:.12f}, {updated_force[2]:.12f})")
    
    print(f"\nLegacy integrator:")
    print(f"  Δp: ({legacy_dp1[0]:.12f}, {legacy_dp1[1]:.12f}, {legacy_dp1[2]:.12f})")
    print(f"  Force: ({legacy_force[0]:.12f}, {legacy_force[1]:.12f}, {legacy_force[2]:.12f})")
    
    # Check signs
    print(f"\n=== FORCE DIRECTION COMPARISON ===")
    for i, axis in enumerate(['X', 'Y', 'Z']):
        expected_sign = np.sign(expected_force[i]) if abs(expected_force[i]) > 1e-12 else 0
        updated_sign = np.sign(updated_force[i]) if abs(updated_force[i]) > 1e-12 else 0
        legacy_sign = np.sign(legacy_force[i]) if abs(legacy_force[i]) > 1e-12 else 0
        
        print(f"{axis}-axis: Expected={expected_sign:+.0f}, Updated={updated_sign:+.0f}, Legacy={legacy_sign:+.0f}")
        
        if expected_sign != 0:
            if updated_sign != expected_sign:
                print(f"  🚨 UPDATED SIGN ERROR in {axis}-axis!")
            if legacy_sign != expected_sign:
                print(f"  🚨 LEGACY SIGN ERROR in {axis}-axis!")
else:
    print("❌ Not enough trajectory points for force analysis")

print("="*60)